In [6]:
import sharepy
import datetime as dt
from datetime import date, timedelta
import pyodbc as pdb
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from sqlalchemy import create_engine 
import urllib
import time
path = 'C:\\Users\\farshid.hosseini\\Documents\\PACCAR\\'

In [2]:
# conect to SharePoint site
s = sharepy.connect("https://mycomplog.sharepoint.com",username="clipservice@ftcli.com", password="updateMySharepoint")
# download the file to a local folder
f = "https://mycomplog.sharepoint.com/BusinessIntelligence/Kent Broadcast Files/KWRBuildPaper_Today.txt"
s.getfile(f)

Authentication successful   


<Response [200]>

In [7]:
build =[]
for line in open('KWRBuildPaper_Today.txt', 'r'):
    item =line.rstrip()
    build.append(item)

In [8]:
BuildPaper = pd.DataFrame(build, columns = ['String'])

In [9]:
#We run the stored procedure to merege data with ADF runing on StremeDF under CLIQ RG
# Database connection - urlib
from sqlalchemy import create_engine 
import urllib

server = 'cliq.database.windows.net'
database = 'CLIQ_PACCAR'
username = 'cliq_admin'
password = 'gfd#2019'
params = urllib.parse.quote_plus(
'DRIVER={ODBC Driver 13 for SQL Server};'+ 
'SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# insert data into staging db table

BuildPaper.to_sql(name ='KWRBuildPaper_Today',schema ='kent', con=engine, if_exists='replace', index=False)

In [5]:
# Step 11) Run data merge SP
with engine.begin() as conn:
    conn.execute("exec [kent].[Update_Broadcast]")